# Monte Carlo Simulation

In [2]:
import numpy as np
from dask.diagnostics import ProgressBar
ProgressBar().register()

In [3]:
def random_walk(s0, mu, sigma, days):
    dt = 1/365.
    prices = np.zeros(days)
    prices[0] = s0
    random_values = np.random.normal(loc=mu * dt, scale=sigma * np.sqrt(dt), size=days-1)
    it = np.nditer(random_values, flags=['f_index'])
    while not it.finished:
        prices[it.index+1] = prices[it.index] * (1 + it[0])
        it.iternext()

    return prices


s0 = 100
K = 100
mu = 0.02
sigma = 0.2
days = 365*10 # 10 years
n = 10000 # number of simulations

Exmaple of normal python code

In [ ]:
# Exmaple of running this locally
result = np.average([max(
    0, 
    np.average(random_walk(s0, mu, sigma, days)) - K
) for i in range(0, n)])

Using Dask delayed

In [ ]:
from dask import delayed
result = delayed(np.average)([delayed(max)(
    0, 
    delayed(np.average)(random_walk(s0, mu, sigma, days)) - K
) for i in range(0, n)])

In [4]:
import time
import dask
import dask.bag as db

def run():
    result = db \
        .from_sequence(range(0, n), npartitions=int(n/100)) \
        .map(lambda _: max(
            0,
            np.average(random_walk(s0, mu, sigma, days)) - K
        )) \
        .to_dataframe(columns=['max']) \
        ['max'].mean()

    return result

In [ ]:
# Dont run this on large graphs!
# run().visualize() 

In [4]:
%%time
print(run().compute())

[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  2min  0.4s
20.768292231314284
CPU times: user 2min 2s, sys: 1.52 s, total: 2min 3s
Wall time: 2min


In [5]:
%%time
from dask.distributed import progress
from dask.distributed import Client, LocalCluster
client = Client('dask-scheduler:8786')
client

CPU times: user 8 ms, sys: 4 ms, total: 12 ms
Wall time: 80.6 ms


In [ ]:
print(run().compute())

In [ ]:

client.close()